<a href="https://colab.research.google.com/github/GuillermoLB/pruebaTFG/blob/main/emocionesfer2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# holaaaaaaaaaaa

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping

def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

GPUs Available:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !cp '/content/drive/MyDrive/Datasets/Fer2013.zip' .
# !unzip -q '/content/Fer2013.zip'

In [ ]:
path = '/content'

In [ ]:
batch_size = 64
img_height = 48
img_width = 48

In [ ]:
# from tensorflow.keras.preprocessing import image_dataset_from_directory

# train_ds = image_dataset_from_directory(
#     '../input/fer2013/train',
#     validation_split=0.2, # 80% para training
#     subset="training",
#     seed=123,
#     image_size=[img_width, img_height],
#     batch_size=batch_size,
#     shuffle=True,
# )

In [ ]:
# val_ds = image_dataset_from_directory(
#     '../input/fer2013/train',
#     validation_split=0.2, # 20% para validation
#     subset="validation",
#     seed=123,
#     image_size=[img_width, img_height],
#     batch_size=batch_size,
#     shuffle=True,
# )

In [ ]:
# class_names = train_ds.class_names
# print(class_names)

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

In [ ]:
generator = ImageDataGenerator(
                            rescale=1. / 255,
                            # rotation_range=45,
                            # width_shift_range=0.2,
                            # height_shift_range=0.2,
                            # horizontal_flip=True,
                            # vertical_flip=True,
                            validation_split=0.2)

test_generator = ImageDataGenerator(rescale=1. / 255)

train_ds = generator.flow_from_directory(path + '/train',
                                        target_size=(img_width, img_height),
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        subset='training')

val_ds = generator.flow_from_directory(path + '/train',
                                        target_size=(img_width, img_height),
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        subset='validation')

test_ds = test_generator.flow_from_directory(path + '/test',
                                            target_size=(img_width, img_height),
                                            class_mode='categorical',
                                            batch_size=batch_size)

FileNotFoundError: ignored

In [ ]:
class_names = train_ds.class_indices
class_names = dict(zip(class_names.values(), class_names.keys()))
print(class_names)

In [ ]:
imgs, labels = next(train_ds)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,10,figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    

In [ ]:
plotImages(imgs)
print(labels)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
early_stopping = EarlyStopping(
    min_delta=0.01, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
from tensorflow.python.keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.python.keras.layers.core import Dropout
num_classes = len(class_names)

model3 = Sequential([
#         layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model2 = Sequential([
#         layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        # layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model = Sequential([
#         layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
        layers.Conv2D(64, 5, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.Conv2D(128, 5, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.2),
        layers.Activation('relu'),
        layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(
                optimizer='adam',
#                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                loss=categorical_crossentropy,
                metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=500
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping]
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();
print("Maximum validation accuracy: {}".format(history_frame['val_accuracy'].max()))
print("Minimum validation loss: {}".format(history_frame['val_loss'].min()))

In [ ]:
imgs, labels = next(test_ds)

In [ ]:
print(model.evaluate(test_ds))

In [ ]:
predictions = model.predict(imgs[:10])
predictions = np.argmax(model.predict(imgs[:10]), axis=-1)
label_predictions = []
for prediction in predictions:
  label_predictions.append(class_names[prediction])
print(label_predictions)
plotImages(imgs[:10])